In [2]:
import autograd.numpy as np
from autograd import grad
from autograd.scipy.integrate import odeint
from autograd.builtins import tuple
from autograd.misc.optimizers import adam
import autograd.numpy.random as npr


N = 30  # Dataset size
D = 2   # Data dimension
max_T = 1.5

# Two-dimensional damped oscillator
def func(y, t0, A):
    return np.dot(y**3, A)

def nn_predict(inputs, t, params):
    for W, b in params:
        outputs = np.dot(inputs, W) + b
        inputs = np.maximum(0, outputs)
    return outputs

def init_nn_params(scale, layer_sizes, rs=npr.RandomState(0)):
    """Build a list of (weights, biases) tuples, one for each layer."""
    return [(rs.randn(insize, outsize) * scale,   # weight matrix
             rs.randn(outsize) * scale)           # bias vector
            for insize, outsize in zip(layer_sizes[:-1], layer_sizes[1:])]

# Define neural ODE model.
def ode_pred(params, y0, t):
    return odeint(nn_predict, y0, t, tuple((params,)), rtol=0.01)

def L1_loss(pred, targets):
    return np.mean(np.abs(pred - targets))

In [3]:
true_y0 = np.array([2., 0.]).T
t = np.linspace(0., max_T, N)
true_A = np.array([[-0.1, 2.0], [-2.0, -0.1]])
true_y = odeint(func, true_y0, t, args=(true_A,))

def train_loss(params):
    pred = ode_pred(params, true_y0, t)
    return L1_loss(pred, true_y)

In [4]:
gradLoss = grad(train_loss)

In [5]:
init_params = init_nn_params(0.1, layer_sizes=[D, 150, D])

In [7]:
gradLoss(init_params)

[(array([[ -6.65766508e-02,   3.22482725e-02,   3.21572868e-02,
           -4.16874490e-02,   1.30732849e-02,   0.00000000e+00,
           -2.47007425e-02,   0.00000000e+00,  -6.78904945e-02,
           -1.51676995e-01,  -1.37221415e-01,  -2.17561539e-02,
            1.02423883e-01,   0.00000000e+00,   7.77178375e-02,
            5.63735269e-02,  -6.26971739e-02,   1.17680926e-02,
            1.00113775e-01,   0.00000000e+00,   0.00000000e+00,
            7.96053051e-02,   1.79738210e-01,   0.00000000e+00,
           -6.83639312e-02,   0.00000000e+00,   5.25459019e-02,
           -2.59453242e-02,  -1.16432869e-01,  -7.26273879e-02,
            6.62737924e-02,  -4.51383413e-03,   0.00000000e+00,
            0.00000000e+00,   0.00000000e+00,  -1.91698318e-02,
           -1.16480056e-01,   1.19847923e-02,   0.00000000e+00,
           -1.54199125e-01,   0.00000000e+00,   0.00000000e+00,
            0.00000000e+00,  -1.83344072e-02,   0.00000000e+00,
            0.00000000e+00,   0.00000000